원문: Moler, Cleve B. [Numerical computing with MATLAB](https://www.mathworks.com/moler/chapters.html). Society for Industrial and Applied Mathematics, 2008.

# 5. 최소자승법

*최소자승법*은 과결정 되어 있거나 부정확하게 명세된 연립방정식의 근을 찾는 데 자주 사용되는 방법이다.  엄밀해를 구하는 대신, 우리는 잔차의 제곱의 합을 최소화하는 것 만을 목표로 한다

최소자승 조건은 중요한 통계적 해석을 가진다. 만일 아래에 깔린 오류 분표에 대한 적당한 확률적 가정이 주어지면, 최소자승법은 매개변수의 *최대 가능도* 추정값을 계산해낼 수 있다. 심지어 확률적 가정이 만족되지 않더라도, 다년간의 경험에 따르면, 최소자승법은 유용한 결과값을 계산해 낸다.

최소 자승 문제를 위한 계산 기법은 직교 행렬 분해를 이용한다.

## 5.1 모델과 곡선적합

매우 자주 최소 자승 문제를 접하는 경우는 곡선 적합이다. $t$가 독립변수이고 우리가 추정하고자 하는 미지의 함수를 $y(t)$로 표시한다고 하자. $m$ 개의 관측값이 있다고 가정하라. 이를테면, $t$의 특정 값에서 $y$ 값이 다음과 같이 관측되었다:
$$y_i=y(t_i), i = 1, \cdots, m$$
아이디어는 $y(t)$ 를 $n$ 개의 *기저함수* 의 선형 조합으로 모델하는 것이다.
$$y(t) \approx \beta_1 \phi_1(t)+\cdots+\beta_n \phi_n(t)$$
해당 *설계 행렬* $X$ 는 $m \times n$ 직사각형 행렬로 각 요소는 다음과 같다.
$$x_{i,j}=\phi_j(t_i)$$
설계 행렬은 보통 행이 열보다 많다. 행렬 벡터 표시로 모델은 다음과 같다.
$$y \approx X \beta $$
기호 $\approx$ 는 "대략 비슷하다"는 뜻이다. 우리는 다음 절에서 더 정확한 의미를 논하겠지만, 우리는 중점을 *최소자승* 추정값에 둔다.
기저함수 \phi_j(t) 는 $t$의 비선형함수일 수도 있지만, 미지의 매개변수 $beta_j$ 는 모델에서 선형적으로 나타난다. 연립방정식
$$X\beta\approx y$$
가 *과결정* 되는 조건은 방정식이 미지수 보다 더 많은 경우이다. 예를 들어 다음과 같이 적용할 수 있다.

In [ ]:
# https://stackoverflow.com/questions/7160162/left-matrix-division-and-numpy-solve
    
import numpy as np
import numpy.linalg as na

t0 = np.linspace(0, 10, 11)
t = t0.reshape((len(t0), 1))
a = 0.5
y_true = a * t
noise = np.random.normal(0.0, 0.5, size=y_true.shape)
y_contaminated = y_true + noise
x, resid, rank, s = na.lstsq(t, y_contaminated)
print('x = %s' % x)
print('resid = %s' % resid)
print('rank = %s' % rank)
print('s = %s' % s)

기저함수는 또한 비선형 매개변수 $\alpha_1, \cdots, \alpha_p$ 를 포함하고 있을 수 있다.  문제가 선형과 비선형 매개변수 모두를 포함하면 *분리가능*이라고 말한다:
$$y(t) \approx \beta_1 \phi_1(t, \alpha) + \cdots + \beta_n \phi_n(t, \alpha) $$
설계 행렬의 요소는 $t$ 와 $\alpha$ 모두에 의존한다.
$$x_{i,j} = \phi_j(t_i, \alpha)$$
분리 가능한 문제는 선형 대수적인 방법과 최적화 기법 가운데 한가지를 복합함으로써 풀 수 있다. 
아래 모델을 참고할 수 있다:
* 직선 : 모델이 $t$ 에 대해 선형이라면, 직선이다:
$$y(t) \approx \beta_1 t + \beta_2 $$
* 다항식 : 계수 $\beta_j$ 가 선형적으로 나타난다. 가장 높은 차수의 계수부터 표시한다:
\begin{align}
\phi_j(t) &= t^{n-j}, j=1, \cdots, n \\
y(t) &\approx \beta_1 t^{n-1} + \cdots + \beta_{n-1}t + \beta_n
\end{align}
``polyfit``으로 최소자승 다항식 곡선 적합이 가능한데 설계 행렬을 만들고 선형 대수적으로 매개변수를 푼다.
* 분수 함수 : 분자의 계수는 선형적으로 나타난다; 분모의 계수는 비선형적으로 나타난다:
\begin{align}
\phi_j(t) &= \frac{t^{n-j}}{\alpha_1 t^{n-1} + \cdots + \alpha_{n-1}t + \alpha_n},\\
y(t) &\approx \frac{\beta_1 t^{n-1} + \cdots + \beta_{n-1}t + \beta_n}{\alpha_1 t^{n-1} + \cdots + \alpha_{n-1}t + \alpha_n}
\end{align}
* 지수 함수 : 감쇠율 $\lambda_j$ 가 비선형적으로 나타난다:
\begin{align}
\phi_j(t) &= e^{-\lambda_j t} \\\
y(t) &\approx \beta_1 e^{-\lambda_1 t} + \cdots + \beta_n e^{-\lambda_n t}
\end{align}
* 로그 선형 : 만일 지수가 하나 뿐이라면, 로그를 취하면 모델이 선형으로 만들지만 적합 조건도 바꾼다.
\begin{align}
y(t) &\approx K e^{\lambda t} \\\
log y &\approx \beta_1 t + \beta_2 \\\
\beta_1 &= \lambda \\\
\beta_2 &= log K
\end{align}
* 정규 분포 : 평균과 표준편차가 비선형적으로 나타난다
\begin{align}
\phi_j(t) &= e^{-(\frac{t-\mu_j}{\sigma_j})^2} \\\
y(t) &\approx \beta_1 e^{-(\frac{t-\mu_1}{\sigma_1})^2} + \cdots + \beta_n e^{-(\frac{t-\mu_n}{\sigma_n})^2}
\end{align}

## 5.2 놂

*잔차* 는 관측값과 모델의 차 이다:
$$r_i=y_i - \sum\limits_{j=1}^n \beta_j \phi_j(t_i, \alpha), i = 1, \cdots, m $$
또는 행렬 벡터 표시로는 다음과 같다.
$$r=y-X(\alpha)\beta$$
우리가 찾기를 원하는 것은 잔차를 가능한 작게 만드는 $\alpha$ 와 $\beta$ 이다. "작다"는 것은 어떤 의미인가? 다른 말로는, 우리가 $\approx$ 기호를 사용할 때 어떤 의미로 사용하는가? 몇가지 가능성이 있다.
* 내삽 : 만일 매개변수의 수가 관측값의 수와 같다면, 우리는 잔차를 0으로 만들 수 있을 지도 모른다. 선혐 문제에서는, 이것이 의미하는 것은 $m=n$ 이고 설계 행렬 $X$는 정방행렬이다. 만일 $X$ 가 특이행렬이 아니라면, $\beta$는 정방 연립 방정식의 해 이다:
$$\beta = X \backslash y$$
* 최소자승 : 잔차의 제곱의 합을 최소화한다.
$$||r||^2 = \sum\limits_{i=1}^m {r_i}^2$$
* 가중 최소 자승 : 만일 어떤 관측값이 다른 것에 비해 더 중요하거나 더 정확하다면 우리는 개별 가중치 $w_j$ 와 개별 관측값을 관련짓고 다음을 최소화 할 수 있다.
$${||r||_w}^2 = \sum\limits_{i=1}^m w_i{r_i}^2$$
예를 들어 만일 $i$번째 관측값의 오차가 대략 $e_i$라면, $w_i = 1 / e_i$ 로 한다.
가중치가 적용되지 않은 최소 자승 문제를 푸는 알고리듬이라면 어떤 것이라도 가중치 문제를 푸는데 사용할 수 있다. 우리는 단순히 $y_i$와 $X$의 $i$번째 행에 $w_i$를 곱하기만 하면 된다. 예를 들어 아래와 같은 방식으로 가능하다.

In [ ]:
import numpy as np
import numpy.linalg as na

t0 = np.linspace(0, 10, 11)
t = t0.reshape((len(t0), 1))
a = 0.5
y_true = a * t

noise = np.random.normal(0.0, 0.5,size=t0.shape)
noise[2] = noise[0] = 0.0
noise = noise.reshape(t.shape)

y_contaminated = y_true + noise
x, resid, rank, s = na.lstsq(t, y_contaminated)

print('x = %s' % x)
print('resid = %s' % resid)
print('rank = %s' % rank)
print('s = %s' % s)

w = np.ones_like(t0)
w[2] = w[0] = 10.0
w_diag = np.matrix(np.diag(w))
t_mat = np.matrix(t)
y_c_mat = np.matrix(y_contaminated)

t_weighted_mat = w_diag * t_mat
y_weighted_mat = w_diag * y_c_mat

xw, residw, rankw, sw = na.lstsq(t_weighted_mat, y_weighted_mat)

print('xw = %s' % xw)
print('residw = %s' % residw)
print('rankw = %s' % rankw)
print('sw = %s' % sw)

* 1-놂 : 잔차의 절대값의 합을 최소화한다:
$$||r||_1 = \sum\limits_{i=1}^m |r_i|$$
이 문제는 선형 계획 문제로 재구성할 수 있지만, 그것은 계산상 최소자승 보다 더 어렵다. 결과로 얻어지는 매개변수는 가짜 자료 또는 *동떨어진 값*의 존재에 덜 민감하다.
* 무한-놂 : 가장 큰 잔차를 최소화 한다:
$$||r||_\infty = \max_{i} |r_i| $$
이것은 체비셰프 적합으로 알려져 있고, 선형 계획법으로 재구성할 수 있다. 체비셰프 적합은 디지털 필터 설계와 수학 함수 라이브러리에 사용되는 근사치를 개발하는데 자주 사용된다.

최적화와 곡선적합 기능을 사용하면 1-놂, 무한-놂 문제를 위한 함수를 포함한다. 우리는 여기서는 최소 자승만 다루기로 한다.

## 5.3 인구 예제

다음 프로그램은 몇가지 다른 선형 모델을 포함한다. 데이터는 미 연방 전체 인구로 미 연방 인구 조사국이 1900 년 부터 2010 년 사이에 조사한 것이다. 단위는 100만명이다.

In [ ]:
t = np.array(range(1900, 2011, 10))
y = np.array([75.995, 91.972, 105.711, 123.203, 131.669, 150.697, 179.323, 203.212, 226.505, 249.633, 281.422, 308.748])
t_y = np.row_stack([t, y])
print('     t           y')
print(t_y.transpose())

할 일은 인구 증가를 모델하고 $t=2020$일 때의 인구를 예측하는 것이다. $t$의 3차 다항식이라면 다음과 같다:
$$y \approx \beta_1 t^3 + \beta_2 t^2 + \beta_3 t + \beta_4$$
4개의 매개변수가 선형적으로 나타나 있다.

In [ ]:
# https://docs.scipy.org/doc/numpy/reference/routines.polynomials.poly1d.html
import numpy as np
import matplotlib.pyplot as plt

polynomial_order = 3

result = np.polyfit(t, y, polynomial_order)
print('result = %s' % result)

t_plot = np.linspace(t.min(), t.max())
y_plot = np.polyval(result, t_plot)

plt.plot(t_plot, y_plot)
plt.plot(t, y, '.')
plt.grid(True)
plt.show()

수치적으로, $t$ 의 제곱을 기저함수로 쓰는 것은 $t$ 가 1900 또는 2000 부근일 때는 좋은 생각이 아니다. 설계 행렬이 좋지 않은 형태가 되고 그 열은 거의 선형 의존이 된다. 훨씬 나은 기저 함수를 만들기 위해 $t$를 평행이동 시킨 후 계수를 곱한다:
$$s = (t-1955)/55$$
새로운 변수는 $-1\le s \le1$ 이고 모델은 다음과 같다.
$$y \approx \beta_1 s^3 + \beta_2 s^2 + \beta_3 s + \beta_4$$
결과로 얻어지는 설계 행렬도 더 나은 상태가 된다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

polynomial_order = 3

scale_factor = (t.max() - t.min()) * 0.5
offset = t.mean()

s = (t - offset) / (scale_factor)

result = np.polyfit(s, y, polynomial_order)
print('result = %s' % result)

s_plot = np.linspace(s.min(), s.max(), 101)
y_plot = np.polyval(result, s_plot)

t_plot = s_plot * scale_factor + offset

plt.plot(t_plot, y_plot)
plt.plot(t, y, '.')
plt.grid(True)
plt.show()

위 그림은 인구 조사 결과를 3차 다항식으로 곡선 적합한 것을 보여 준다. 2020년 까지의 외삽은 합리적인 것으로 보인다. 다항식의 차수도 다양하게 시도해 볼 수 있다. $||r||$이 감소한다는 관점에서 차수가 증가함에 따라 곡선 적합이 더 정확해진다. 왜냐하면 관찰값 사이와 바깥의 변화가 더 커지기 때문이다.

내삽하는 방법으로 ``spline`` 등 다른 방법들도 있다.
가장 중요한 질문인 "어떤 모델이 최선인가?" 는 여러분 스스로 답을 찾아야 한다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as si

polynomial_order = 3

scale_factor = (t.max() - t.min()) * 0.5
offset = t.mean()

s = (t - offset) / (scale_factor)

result_polyfit = np.polyfit(s, y, polynomial_order)
print('result_polyfit = %s' % result_polyfit)

# https://stackoverflow.com/questions/12935098/how-to-plot-line-polygonal-chain-with-numpy-scipy-matplotlib-with-minimal-smoo
result_pchip = si.pchip(s, y)

s_plot = np.linspace(s.min(), s.max(), 101)
y_polyfit_plot = np.polyval(result_polyfit, s_plot)
y_pchip_plot = result_pchip(s_plot)

t_plot = s_plot * scale_factor + offset

plt.plot(t_plot, y_polyfit_plot, label='polyfit')
plt.plot(t_plot, y_pchip_plot, label='pchip')
plt.plot(t, y, '.')
plt.legend(loc=0)
plt.grid(True)
plt.show()

## 5.4 하우스홀더 거울상

하우스홀더 거울상은 일종의 행렬 변환으로, 알려져 있는 가장 강력하고 유연한 수치 알고리듬의 기반이 된다. 우리는 하우스홀더 거울상을 사용하여 이 장에서는 최소자승해를 구하고 이후의 장에서는 행렬의 고유치와 특이값 문제를 풀 것이다.
형식상으로 하우스홀더 거울상은 아래와 같은 행렬이다.

$$H = I - \rho u u^T $$

여기서 $u$는 어떤 0이 아닌 벡터이고 $\rho = 2/||u||^2$ 이다. $uu^T$ 는 행렬로 랭크는 1로 모든 열은 $u$의 배수이고 모든 행은 $u^T$의 배수이다. 결과로 얻어지는 행렬  $H$ 은 대칭이고 직교이므로 다음 성질을 가진다.
\begin{align}
H^T &=H\\\
H^T H &= H^2 = I
\end{align}
실제로는 행렬 $H$ 는 형성하지 않는다. 대신 $H$를 어떤 벡터 $x$ 는 다음과 같이 계산한다.
\begin{align}
\tau &= \rho u^T x\\\
H x &= x - \tau u
\end{align}

기하적으로는 벡터 $x$가 $u$에 투영된 후 그 투영된 벡터의 두배를 $x$ 로 부터 뺀다.

아래 코드는 하우스홀더 거울상을 보여 준다. 벡터 $u$ 와 직선 $u_{\perp}$는 직교한다. 두 벡터 $x$와 $y$ 의 거울상 $Hx$ 와 $Hy$ 를 변환 H 로 구한다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.matrix([[3, 2]]).T
y = np.matrix([[7, -4]]).T

# find u bisecting x and x axis
x_complex = complex(x[0, 0], x[1, 0])
x_angle_rad = np.angle(x_complex)
x_len = abs(x_complex)
u_angle_rad = 0.5 * x_angle_rad
u = 2 * x_len * np.matrix([[np.cos(u_angle_rad), np.sin(u_angle_rad)]]).T

# Householder reflection
rho = (2 / (u.T * u))[0, 0]

tau_x = (rho * u.T * x)[0, 0]
Hx = x - tau_x * u

tau_y = (rho * u.T * y)[0, 0]
Hy = y - tau_y * u

# prepare data to draw figure
x_arrow = [[0, 0], x.T.tolist()[0]]
y_arrow = [[0, 0], y.T.tolist()[0]]
u_arrow = [[0, 0], u.T.tolist()[0]]
u_ortho_arrow = np.array([[-u[1, 0], u[0, 0]],
                          [u[1, 0], -u[0, 0]]]) * 1.6
Hx_arrow = [[0, 0], Hx.T.tolist()[0]]
Hy_arrow = [[0, 0], Hy.T.tolist()[0]]

# prepare axis
ax = plt.gca()

# clear axis
ax.cla()

# draw arrows
ax.arrow(x_arrow[0][0], x_arrow[0][1], x_arrow[1][0], x_arrow[1][1], head_width = 0.5, head_length=0.5)
ax.arrow(y_arrow[0][0], y_arrow[0][1], y_arrow[1][0], y_arrow[1][1], head_width = 0.5, head_length=0.5)
ax.arrow(u_arrow[0][0], u_arrow[0][1], u_arrow[1][0], u_arrow[1][1], head_width = 0.5, head_length=0.5)
ax.arrow(Hx_arrow[0][0], Hx_arrow[0][1], Hx_arrow[1][0], Hx_arrow[1][1], head_width = 0.5, head_length=0.5)
ax.arrow(Hy_arrow[0][0], Hy_arrow[0][1], Hy_arrow[1][0], Hy_arrow[1][1], head_width = 0.5, head_length=0.5)

# draw texts
offset = 0.5
ax.text(x_arrow[1][0]+offset, x_arrow[1][1], 'x')
ax.text(y_arrow[1][0]+offset, y_arrow[1][1], 'y')
ax.text(u_arrow[1][0]+offset, u_arrow[1][1], 'u')

ax.text(Hx_arrow[1][0]-offset*3, Hx_arrow[1][1], 'Hx')
ax.text(Hy_arrow[1][0]-offset*3, Hy_arrow[1][1], 'Hy')

ax.text(u_ortho_arrow[0, 0]*0.5, u_ortho_arrow[0, 1]*0.5, '$u_\perp$')

# draw line orthogonal to u
plt.plot(u_ortho_arrow[:,0], u_ortho_arrow[:, 1])

# adjust aspect ratio
plt.axis('equal')

# set axis limits
plt.xlim(-10, 10)
plt.ylim(-8, 8)

# draw grid
plt.grid(True)

# present plot
plt.show()

해당 행렬은 어떠한 벡터라도 직선 $u_{\perp}$ 에 대칭하는 거울상으로 변환한다. 어떤 벡터 $x$와 그 거울상 $Hx$ 의 중점인 벡터
$$x-(\tau / 2) u$$
는 직선 $u_{\perp}$ 위에 있다. 2 보다 큰 차원에서는, $u_{\perp}$가 정의하는 벡터 $u$에 수직인 (초)평면이다.

만일 $u$가 $x$와 좌표축 사이의 각을 이등분한다면, 결과로 얻어지는 $Hx$는 해당 좌표축 위에 그려진다. 다시말해, $x$ 의 요소가 하나를 제외하고는 0이 된다. 더 나아가서 $H$가 수직이므로 그 길이가 보존된다. 결과적으로 $Hx$의 0이 아닌 요소는 $\pm ||x||$ 이다.